In [1]:
import numpy as np
import pandas as pd

### Get positive negative data

In [88]:
from scar import load_scar
x, y, s = load_scar()

In [89]:
s[s==1].shape

(1644,)

### Labeling

In [50]:
class LabelingMechanism():
    def __init__(self,
                 propensity_attributes,
                 propensity_attributes_signs,
                 min_prob=0.2,
                 max_prob=0.8):
        assert len(propensity_attributes) == len(propensity_attributes_signs), "size of attributes and signs must be same"
        self.propensity_attributes = np.array(propensity_attributes)
        self.nb_propensity_attributes = len(propensity_attributes)
        self.propensity_attributes_signs = np.array(propensity_attributes_signs)
        self.min_prob = min_prob
        self.max_prob = max_prob
        
    def fit(self, x):
        x_e = x[:, self.propensity_attributes] * self.propensity_attributes_signs
        self.minx = x_e.min(0)
        self.maxx = x_e.max(0)
        
    def propensity_score(self, x):
        x_e = x[:, self.propensity_attributes] * self.propensity_attributes_signs
        scaled = self.min_prob + (x_e - self.minx) / (self.maxx - self.minx) * (self.max_prob - self.min_prob)
        return (scaled**(1 / self.nb_propensity_attributes)).prod(1)

In [104]:
lm = LabelingMechanism([1,1],[1,1], min_prob=0, max_prob=0.2)
lm.fit(x)
score = lm.propensity_score(x)

In [105]:
s = (y * (np.random.uniform(size=x.shape[0]) < score)).astype(int)

### Visualize created data

In [81]:
from utils import visualize_pn_data
visualize_pn_data(x, y)

In [106]:
from utils import visualize_pu_data
visualize_pu_data(x, s)